In [1]:
!pip install -q torch transformers accelerate bitsandbytes langchain sentence-transformers faiss-cpu openpyxl pacmap datasets langchain-community ragatouille

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 58.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd

In [3]:
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
from datasets import Dataset

In [4]:
df = pd.read_csv('/kaggle/input/resume-dataset/UpdatedResumeDataSet.csv')


In [5]:
# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)

In [6]:
from langchain.docstore.document import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["Resume"], metadata={"Category": doc["Category"]}) for doc in tqdm(dataset)
]

  0%|          | 0/962 [00:00<?, ?it/s]

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# We use a hierarchical list of separators specifically tailored for splitting Markdown documents
# This list is taken from LangChain's MarkdownTextSplitter class
MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # The maximum number of characters in a chunk: we selected this value arbitrarily
    chunk_overlap=100,  # The number of characters to overlap between chunks
    add_start_index=True,  # If `True`, includes chunk's start index in metadata
    strip_whitespace=True,  # If `True`, strips whitespace from the start and end of every document
    separators=MARKDOWN_SEPARATORS,
)

docs_processed = []
for doc in RAW_KNOWLEDGE_BASE:
    docs_processed += text_splitter.split_documents([doc])

In [8]:
# docs_processed

In [9]:
from sentence_transformers import SentenceTransformer

# To get the value of the max sequence_length, we will query the underlying `SentenceTransformer` object used in the RecursiveCharacterTextSplitter
print(f"Model's maximum sequence length: {SentenceTransformer('thenlper/gte-small').max_seq_length}")

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("thenlper/gte-small")
lengths = [len(tokenizer.encode(doc.page_content)) for doc in tqdm(docs_processed)]

2025-04-27 09:33:52.237509: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745746432.425054      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745746432.478886      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model's maximum sequence length: 512


  0%|          | 0/3893 [00:00<?, ?it/s]

In [23]:

EMBEDDING_MODEL_NAME = "thenlper/gte-small"

In [15]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
    docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
)

2025-04-27 09:35:03.155011: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745746503.176734     164 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745746503.183424     164 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Chunks:   0%|          | 0/10 [00:00<?, ?it/s]

In [16]:
KNOWLEDGE_VECTOR_DATABASE

In [1]:
retriever = KNOWLEDGE_VECTOR_DATABASE.as_retriever()

NameError: name 'KNOWLEDGE_VECTOR_DATABASE' is not defined

In [17]:
# Embed a user query in the same space
user_query = "retrive top 5 doc from data science catogery"
query_vector = embedding_model.embed_query(user_query)

2025-04-27 09:35:25.973456: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745746525.994874     184 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745746526.001482     184 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=5)


2025-04-27 09:35:34.696058: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745746534.717628     204 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745746534.724201     204 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
import pprint

In [21]:
pprint.pp(retrieved_docs)

[Document(id='660577a3-3224-490f-9171-c7996c67a8e4', metadata={'Category': 'Data Science', 'start_index': 3225}, page_content='â\x9e\x94 Sentiment Analysis   Python, NLP, Apache Spark service in IBM Bluemix.\r\nThis project is highly emphasis on tweets from Twitter data were taken for mobile networks service provider to do a sentiment analysis and analyze whether the expressed opinion was positive, negative or neutral, capture the emotions of the tweets and comparative analysis.\r\n\r\nQuantifiable Results:\r\nâ\x88\x92 Mentored 7-12 Data Science Enthusiast each year that have all since gone on to graduate school in Data Science and Business Analytics.\r\nâ\x88\x92 Reviewed and evaluated 20-40 Research Papers on Data Science for one of the largest Data Science Conference called Data Science Congress by Aegis School of Business Mumbai.\r\nâ\x88\x92 Heading a solution providing organization called Data Science Delivered into Aegis school of Data Science Mumbai and managed 4-5 live projec

In [25]:
# Create FAISS index
vector_db = FAISS.from_documents(KNOWLEDGE_VECTOR_DATABASE, EMBEDDING_MODEL_NAME)

# Save the index locally (optional)
vector_db.save_local("resume_faiss_index")

TypeError: 'FAISS' object is not iterable

In [17]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

READER_MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)

READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not in

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Device set to use cuda:0


In [18]:
READER_LLM("how many resumes are in Data Science Category?")

[{'generated_text': '\n\nI am interested in learning more about the Data Science category on your platform. Can you provide me with some statistics regarding the number of resumes available in this category and any trends or patterns that have emerged from analyzing these resumes? Additionally, could you suggest any best practices for optimizing a resume to stand out in this competitive field?'}]

In [24]:
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """
1. Analyze all the resumes to assess its alignment with the provided job description.
2. Stick strictly to the facts provided in the resumes and job description. Never make assumptions or invent information.
3. Select only the top 20 resumes based on their alignment with the job description. Ensure these are ranked clearly by relevance (1st: Most Suitable, 2nd: Next Best, 3rd: Final Option).
4. Evaluate key areas, including:
    Education: Does the candidate meet the educational requirements?
    Experience: Is their work experience relevant and sufficient for the role?
    Skills: Are the technical, soft, and domain-specific skills present?
5. For each resume, provide:
    Strengths: Key qualifications that match the role.
    Gaps: Areas where the candidate does not meet the criteria.
6. Summarize why these top 20 resumes were chosen, focusing on their alignment with the job requirements.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)
print(RAG_PROMPT_TEMPLATE)

<|system|>

1. Analyze all the resumes to assess its alignment with the provided job description.
2. Stick strictly to the facts provided in the resumes and job description. Never make assumptions or invent information.
3. Select only the top 20 resumes based on their alignment with the job description. Ensure these are ranked clearly by relevance (1st: Most Suitable, 2nd: Next Best, 3rd: Final Option).
4. Evaluate key areas, including:
    Education: Does the candidate meet the educational requirements?
    Experience: Is their work experience relevant and sufficient for the role?
    Skills: Are the technical, soft, and domain-specific skills present?
5. For each resume, provide:
    Strengths: Key qualifications that match the role.
    Gaps: Areas where the candidate does not meet the criteria.
6. Summarize why these top 20 resumes were chosen, focusing on their alignment with the job requirements.</s>
<|user|>
Context:
{context}
---
Now here is the question you need to answer.

Qu

In [25]:
context = """The following is the job description: 'Job Title: Data Scientist

Job Duties:
Developing and deploying machine learning models to solve business problems. 
Cleaning, processing, and analyzing large datasets to extract actionable insights.
Collaborating with cross-functional teams (engineering, product, marketing) to implement
data-driven solutions. Designing and maintaining data pipelines for efficient
data collection and processing. Conducting statistical analysis and A/B testing to 
measure the impact of business decisions. Building predictive models and recommendation
systems to enhance user experience. Creating data visualizations and dashboards to 
communicate findings to stakeholders. Staying updated with the latest advancements in 
AI/ML and applying best practices. Automating data workflows to improve
efficiency and scalability. Other ad-hoc data-related tasks as required.
"""

In [26]:
final_prompt = RAG_PROMPT_TEMPLATE.format(question="Select and Retrive only the top 20 resumes based on their alignment with the job description?", context=context)

# Redact an answer
answer = READER_LLM(final_prompt)[0]["generated_text"]
print(answer)

To select and retrieve the top 20 resumes based on their alignment with the job description, follow these steps:

1. Analyze all the resumes to assess their alignment with the provided job description. Look for candidates who have experience in developing and deploying machine learning models, cleaning and processing large datasets, collaborating with cross-functional teams, designing and maintaining data pipelines, conducting statistical analysis and A/B testing, building predictive models and recommendation systems, creating data visualizations and dashboards, staying updated with the latest advancements in AI/ML, automating data workflows, and performing other ad-hoc data-related tasks as required.

2. Stick strictly to the facts provided in the resumes and job description. Do not make assumptions or invent information.

3. Rank the resumes clearly by relevance (1st: Most Suitable, 2nd: Next Best, 3rd: Final Option). This will help you easily identify the top 20 resumes based on the